In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

%debug

ERROR:root:No traceback has been produced, nothing to debug.


In [4]:
import sys
sys.path.insert(0, "../../code-previous")

In [5]:
import pandas as pd
import numpy as np
import networkx as nx
import sklearn.feature_extraction as skfeatures
import time
import os

import operator
import pickle
import string
import swifter
import nltk
stopwords = nltk.corpus.stopwords.words('english')


import plotly
import plotly.plotly as py
import plotly.graph_objs as go
plotly.tools.set_credentials_file(username='sreejith_s', api_key='rcet0rTqFCc3WeOzWTjv')



from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter

from unicodedata import category
from tqdm import tqdm
tqdm.pandas(desc="Progress")

import seaborn as sns
sns.set_style('white')
import matplotlib.pylab as plt

import WOSutilities as wosutil

path2rawdata = '/mnt/disks/vault/wos2017-parsed/'

In [6]:
disciplines = ["Architecture","Art","Humanities, Multidisciplinary","Folklore","Medieval & Renaissance Studies","Asian Studies","Classics","Dance","Film, Radio, Television","History","History & Philosophy of Science","Literary Theory & Criticism","Literary Reviews","Literature","Literature, African, Australian, Canadian","Literature, American","Literature, British Isles","Literature, German, Dutch, Scandinavian","Literature, Romance","Literature, Slavic","Poetry","Music","Philosophy","Religion","Theater","Agriculture, Dairy & Animal Science","Agricultural Engineering","Agricultural Economics & Policy","Agriculture, Multidisciplinary","Agronomy","Horticulture","Soil Science","Allergy","Anatomy & Morphology","Anesthesiology","Anthropology","Audiology & Speech-Language Pathology","Behavioral Sciences","Biochemical Research Methods","Biochemistry & Molecular Biology","Biodiversity Conservation","Biophysics","Biotechnology & Applied Microbiology","Cardiac & Cardiovascular Systems","Peripheral Vascular Disease","Cell & Tissue Engineering","Cell Biology","Dentistry, Oral Surgery & Medicine","Dermatology","Developmental Biology","Emergency Medicine","Andrology","Endocrinology & Metabolism","Entomology","Ecology","Environmental Sciences","Environmental Studies","Evolutionary Biology","Fisheries","Food Science & Technology","Forestry","Gastroenterology & Hepatology","Critical Care Medicine","Primary Health Care","Medicine, General & Internal","Genetics & Heredity","Geriatrics & Gerontology","Gerontology","Health Care Sciences & Services","Health Policy & Services","Hematology","Immunology","Infectious Diseases","Integrative & Complementary Medicine","Medicine, Legal","Biology","Limnology","Marine & Freshwater Biology","Mathematical & Computational Biology","Medical Ethics","Medical Informatics","Medical Laboratory Technology","Microbiology","Mycology","Clinical Neurology","Neurosciences","Neuroimaging","Nursing","Nutrition & Dietetics","Obstetrics & Gynecology","Oncology","Ophthalmology","Orthopedics","Otorhinolaryngology","Paleontology","Parasitology","Pathology","Pediatrics","Chemistry, Medicinal","Pharmacology & Pharmacy","Physiology","Plant Sciences","Psychiatry","Public, Environmental & Occupational Health","Radiology, Nuclear Medicine & Medical Imaging","Rehabilitation","Reproductive Biology","Medicine, Research & Experimental","Respiratory System","Rheumatology","Sport Sciences","Substance Abuse","Surgery","Toxicology","Transplantation","Tropical Medicine","Urology & Nephrology","Veterinary Sciences","Virology","Ornithology","Zoology","Astronomy & Astrophysics","Chemistry, Applied","Chemistry, Multidisciplinary","Chemistry, Analytical","Chemistry, Inorganic & Nuclear","Chemistry, Organic","Chemistry, Physical","Crystallography","Electrochemistry","Geochemistry & Geophysics","Geology","Geosciences, Multidisciplinary","Mathematics, Applied","Mathematics, Interdisciplinary Applications","Mathematics","Statistics & Probability","Meteorology & Atmospheric Sciences","Mineralogy","Mining & Mineral Processing","Oceanography","Optics","Geography, Physical","Physics, Applied","Physics, Fluids & Plasmas","Physics, Atomic, Molecular & Chemical","Physics, Multidisciplinary","Physics, Condensed Matter","Physics, Nuclear","Physics, Particles & Fields","Physics, Mathematical","Polymer Science","Thermodynamics","Water Resources","Nanoscience & Nanotechnology","Logic","Multidisciplinary Sciences","Acoustics","Automation & Control Systems","Computer Science, Artificial Intelligence","Computer Science, Cybernetics","Computer Science, Hardware & Architecture","Computer Science, Information Systems","Computer Science, Interdisciplinary Applications","Computer Science, Software Engineering","Computer Science, Theory & Methods","Construction & Building Technology","Energy & Fuels","Engineering, Aerospace","Engineering, Multidisciplinary","Engineering, Biomedical","Engineering, Environmental","Engineering, Chemical","Engineering, Industrial","Engineering, Manufacturing","Engineering, Marine","Engineering, Civil","Engineering, Ocean","Engineering, Petroleum","Engineering, Electrical & Electronic","Engineering, Mechanical","Engineering, Geological","Ergonomics","Imaging Science & Photographic Technology","Information Science & Library Science","Instruments & Instrumentation","Materials Science, Paper & Wood","Materials Science, Ceramics","Materials Science, Multidisciplinary","Materials Science, Biomaterials","Materials Science, Characterization & Testing","Materials Science, Coatings & Films","Materials Science, Composites","Materials Science, Textiles","Mechanics","Metallurgy & Metallurgical Engineering","Microscopy","Nuclear Science & Technology","Operations Research & Management Science","Remote Sensing","Robotics","Spectroscopy","Telecommunications","Transportation","Transportation Science & Technology","Archaeology","Area Studies","Social Sciences, Biomedical","Business","Business, Finance","Economics","Industrial Relations & Labor","Management","Communication","Criminology & Penology","Cultural Studies","Demography","Education & Educational Research","Education, Scientific Disciplines","Education, Special","Ethnic Studies","Family Studies","Geography","Law","Political Science","International Relations","Linguistics","Language & Linguistics","Social Sciences, Mathematical Methods","Psychology, Biological","Psychology, Clinical","Psychology, Educational","Psychology, Developmental","Psychology, Applied","Psychology","Psychology, Multidisciplinary","Psychology, Psychoanalysis","Psychology, Mathematical","Psychology, Experimental","Psychology, Social","Planning & Development","Public Administration","Social Issues","Ethics","History of Social Sciences","Hospitality, Leisure, Sport & Tourism","Social Sciences, Interdisciplinary","Social Work","Sociology","Urban Studies","Women's Studies"]

In [7]:
# we need to load articles and categories
article_df = wosutil.load_wos_data(name = 'article', 
                                   path2rawdata = path2rawdata,
                                   year_list = None, 
                                   columns = ['ArticleID', 'PubYear', 'Doctypes', 'Title','Journal', 'Journal Abbreviation ISO'], 
                                   dropna = ['ArticleID', 'PubYear','Title'], 
                                   duplicate_subset = ['ArticleID'],
                                   isindict = {'Doctypes':np.sort(['Article','Letter','Review','Note'])}, 
                                   verbose = 50)

# select all categories for all the aricles published
# category_list = np.array(['Physics, Applied'])  

category_list = np.array(["Architecture","Art","Humanities, Multidisciplinary","Folklore","Medieval & Renaissance Studies","Asian Studies","Classics","Dance","Film, Radio, Television","History","History & Philosophy of Science","Literary Theory & Criticism","Literary Reviews","Literature","Literature, African, Australian, Canadian","Literature, American","Literature, British Isles","Literature, German, Dutch, Scandinavian","Literature, Romance","Literature, Slavic","Poetry","Music","Philosophy","Religion","Theater","Agriculture, Dairy & Animal Science","Agricultural Engineering","Agricultural Economics & Policy","Agriculture, Multidisciplinary","Agronomy","Horticulture","Soil Science","Allergy","Anatomy & Morphology","Anesthesiology","Anthropology","Audiology & Speech-Language Pathology","Behavioral Sciences","Biochemical Research Methods","Biochemistry & Molecular Biology","Biodiversity Conservation","Biophysics","Biotechnology & Applied Microbiology","Cardiac & Cardiovascular Systems","Peripheral Vascular Disease","Cell & Tissue Engineering","Cell Biology","Dentistry, Oral Surgery & Medicine","Dermatology","Developmental Biology","Emergency Medicine","Andrology","Endocrinology & Metabolism","Entomology","Ecology","Environmental Sciences","Environmental Studies","Evolutionary Biology","Fisheries","Food Science & Technology","Forestry","Gastroenterology & Hepatology","Critical Care Medicine","Primary Health Care","Medicine, General & Internal","Genetics & Heredity","Geriatrics & Gerontology","Gerontology","Health Care Sciences & Services","Health Policy & Services","Hematology","Immunology","Infectious Diseases","Integrative & Complementary Medicine","Medicine, Legal","Biology","Limnology","Marine & Freshwater Biology","Mathematical & Computational Biology","Medical Ethics","Medical Informatics","Medical Laboratory Technology","Microbiology","Mycology","Clinical Neurology","Neurosciences","Neuroimaging","Nursing","Nutrition & Dietetics","Obstetrics & Gynecology","Oncology","Ophthalmology","Orthopedics","Otorhinolaryngology","Paleontology","Parasitology","Pathology","Pediatrics","Chemistry, Medicinal","Pharmacology & Pharmacy","Physiology","Plant Sciences","Psychiatry","Public, Environmental & Occupational Health","Radiology, Nuclear Medicine & Medical Imaging","Rehabilitation","Reproductive Biology","Medicine, Research & Experimental","Respiratory System","Rheumatology","Sport Sciences","Substance Abuse","Surgery","Toxicology","Transplantation","Tropical Medicine","Urology & Nephrology","Veterinary Sciences","Virology","Ornithology","Zoology","Astronomy & Astrophysics","Chemistry, Applied","Chemistry, Multidisciplinary","Chemistry, Analytical","Chemistry, Inorganic & Nuclear","Chemistry, Organic","Chemistry, Physical","Crystallography","Electrochemistry","Geochemistry & Geophysics","Geology","Geosciences, Multidisciplinary","Mathematics, Applied","Mathematics, Interdisciplinary Applications","Mathematics","Statistics & Probability","Meteorology & Atmospheric Sciences","Mineralogy","Mining & Mineral Processing","Oceanography","Optics","Geography, Physical","Physics, Applied","Physics, Fluids & Plasmas","Physics, Atomic, Molecular & Chemical","Physics, Multidisciplinary","Physics, Condensed Matter","Physics, Nuclear","Physics, Particles & Fields","Physics, Mathematical","Polymer Science","Thermodynamics","Water Resources","Nanoscience & Nanotechnology","Logic","Multidisciplinary Sciences","Acoustics","Automation & Control Systems","Computer Science, Artificial Intelligence","Computer Science, Cybernetics","Computer Science, Hardware & Architecture","Computer Science, Information Systems","Computer Science, Interdisciplinary Applications","Computer Science, Software Engineering","Computer Science, Theory & Methods","Construction & Building Technology","Energy & Fuels","Engineering, Aerospace","Engineering, Multidisciplinary","Engineering, Biomedical","Engineering, Environmental","Engineering, Chemical","Engineering, Industrial","Engineering, Manufacturing","Engineering, Marine","Engineering, Civil","Engineering, Ocean","Engineering, Petroleum","Engineering, Electrical & Electronic","Engineering, Mechanical","Engineering, Geological","Ergonomics","Imaging Science & Photographic Technology","Information Science & Library Science","Instruments & Instrumentation","Materials Science, Paper & Wood","Materials Science, Ceramics","Materials Science, Multidisciplinary","Materials Science, Biomaterials","Materials Science, Characterization & Testing","Materials Science, Coatings & Films","Materials Science, Composites","Materials Science, Textiles","Mechanics","Metallurgy & Metallurgical Engineering","Microscopy","Nuclear Science & Technology","Operations Research & Management Science","Remote Sensing","Robotics","Spectroscopy","Telecommunications","Transportation","Transportation Science & Technology","Archaeology","Area Studies","Social Sciences, Biomedical","Business","Business, Finance","Economics","Industrial Relations & Labor","Management","Communication","Criminology & Penology","Cultural Studies","Demography","Education & Educational Research","Education, Scientific Disciplines","Education, Special","Ethnic Studies","Family Studies","Geography","Law","Political Science","International Relations","Linguistics","Language & Linguistics","Social Sciences, Mathematical Methods","Psychology, Biological","Psychology, Clinical","Psychology, Educational","Psychology, Developmental","Psychology, Applied","Psychology","Psychology, Multidisciplinary","Psychology, Psychoanalysis","Psychology, Mathematical","Psychology, Experimental","Psychology, Social","Planning & Development","Public Administration","Social Issues","Ethics","History of Social Sciences","Hospitality, Leisure, Sport & Tourism","Social Sciences, Interdisciplinary","Social Work","Sociology","Urban Studies","Women's Studies"])

category_df = wosutil.load_wos_data(name = 'category', 
                                    path2rawdata = path2rawdata,
                                    year_list = None,
                                    columns = ['ArticleID', 'Type', 'Label'], 
                                    dropna = ['ArticleID'], 
                                    duplicate_subset = None,
                                    isindict = {'Label':category_list},
                                    verbose = 100)

50
100
150
200
250
300
350
400
450
Final DF Shape (39171585, 6)
100
200
300
400
Final DF Shape (24657695, 3)


In [8]:
data = None
data = category_df.merge(article_df, 
                         how = 'inner',
                         on = 'ArticleID')

In [9]:
data = data.drop_duplicates()
data.to_pickle("/mnt/disks/vault/analysis-data/interdisciplinary-novelty-analysis/corrected-entropy-analysis/all_data.pql")

In [11]:
len(data['Label'].unique())

135

In [13]:
len(category_df['Label'].unique())

137

In [18]:
len(data)

17827508

In [15]:
'Physics, Applied' in category_df['Label'].unique()

True

In [17]:
category_df['Label'].unique()

array(['Biochemistry & Molecular Biology',
       'Cardiac & Cardiovascular Systems', 'Engineering, Aerospace',
       'Biophysics', 'Crystallography', 'Energy & Fuels',
       'Endocrinology & Metabolism',
       'Biotechnology & Applied Microbiology', 'Political Science',
       'Computer Science, Hardware & Architecture', 'Immunology',
       'Physiology', 'Pharmacology & Pharmacy', 'Oncology',
       'Imaging Science & Photographic Technology',
       'Materials Science, Multidisciplinary',
       'Instruments & Instrumentation', 'Law', 'Forestry', 'Microbiology',
       'Psychology, Multidisciplinary', 'Pediatrics', 'Orthopedics',
       'Obstetrics & Gynecology', 'Biodiversity Conservation',
       'Food Science & Technology', 'Pathology', 'Mycology',
       'Dentistry, Oral Surgery & Medicine', 'Ophthalmology',
       'Hematology', 'Social Sciences, Interdisciplinary', 'Entomology',
       'Cell Biology', 'Nutrition & Dietetics', 'Urban Studies',
       'Biochemical Research Met